In [ ]:
from db.db import backup_database

backup_destination = r"N:\PPDO\BS\Records Department\Archive_Data"
backup_database(backup_dir=backup_destination, compress=True)

In [ ]:
from sqlalchemy.orm import Session
from sqlalchemy import or_
import pandas as pd
import numpy as np
from db import get_db_engine
from db.models import FileLocation, File, FileContent
from text_extraction.pdf_extraction import PDFTextExtractor
from embedding.minilm import MiniLMEmbedder
from utils import extract_server_dirs, build_file_path
server_mount = r"N:\PPDO\Records"
target_filepath = r"N:\PPDO\Records\23xx   Crown College (College 3)\2318\2318\H - Submittals\Lumber_ProductData.pdf"
target_location = r"N:\PPDO\Records\37xx   Oakes College\3701\3701"


pdf_extractor = PDFTextExtractor()
embedder = MiniLMEmbedder()

text = pdf_extractor(target_filepath)
embeddings = embedder.encode(text)
target_loc_dirs = extract_server_dirs(full_path=target_location, base_mount=server_mount)
engine = get_db_engine()

target_sim_distance = lambda file_embeds: 1 - np.dot(file_embeds, embeddings)
build_local_path = lambda row: build_file_path(base_mount=server_mount, server_dir=row['file_server_directories'], filename=row['filename'])

with Session(engine) as db_session:
    files_located_in_dir = or_(
        FileLocation.file_server_directories == target_loc_dirs,
        FileLocation.file_server_directories.startswith(target_loc_dirs + '/')
    )

    query = db_session.query(FileLocation, FileContent)

In [ ]:
# This cell is to backfill the text_length column in FileContent for existing rows where source_text is present.

from db.models import FileContent
from db.db import get_db_engine
from sqlalchemy.orm import sessionmaker

# Create a session
engine = get_db_engine()
Session = sessionmaker(bind=engine)
session = Session()

# Query all FileContent rows where text_length is null and source_text is not null
to_update = session.query(FileContent).filter(
    (FileContent.text_length.is_(None)) & (FileContent.source_text.isnot(None))
).all()

print(f"Found {len(to_update)} rows to backfill.")

for fc in to_update:
    fc.text_length = len(fc.source_text)

session.commit()
print("Backfill complete.")

session.close()

In [ ]:
import os
import dotenv
import pandas as pd
dotenv.load_dotenv()

app_dsn = (
    f"postgresql://{os.getenv('APP_DB_USERNAME')}:{os.getenv('APP_DB_PASSWORD')}"
    f"@{os.getenv('APP_DB_HOST')}:{os.getenv('APP_DB_PORT')}/{os.getenv('APP_DB_NAME')}"
)

g22_query = """SELECT *
FROM file_locations
WHERE file_server_directories LIKE '%/G22%'"""

# use psycopg3 to execute the query and load results into a DataFrame
g22_df = pd.read_sql(g22_query, app_dsn)

ModuleNotFoundError: No module named 'psycopg2'